# Download Wikipedia data\


* Korean Wikipedia data file url 

  https://dumps.wikimedia.org/kowiki/
  
   - Used data : latest/kowiki-latest-pages-articles-multistream.xml.bz2


* Wikipedia data extractor

  https://github.com/attardi/wikiextractor.git
  

### Download korean wiki data

In [ ]:
from urllib.request import urlretrieve

file_url = 'https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles-multistream.xml.bz2'
file_path =  'data/kowiki-latest-pages-articles-multistream.xml_new.bz2'
# download the file'
urlretrieve(file_url,file_path)


### Extract Wiki data

  Takes approximately 1hour.
  https://github.com/attardi/wikiextractor.git

In [ ]:
import subprocess

extractor_path = '/home/tony/PycharmProjects/wikiextractor/WikiExtractor.py'
extract_path = 'wiki_kor_new'

subprocess.call(['python3',extractor_path,file_path, "-o={}".format(extract_path)])

# Preprocess the Text

* 1-1. Remove blank lines.
* 1-2. Remove <doc id ... line and its next line (title of an article).
* 1-3. Replace </doc> line with a blank line.
* 2-1. Remove spaces at the end of each line.
* 2-2. Break line at each 。, but not at 。」 or 。）, position.
* 2-3. Remove spaces at the head of each line.
* 3. Remove lines with the head 。(these lines are not meaningful).
* 4. Convert upper case characters to lower case ones.



In [ ]:
# Input file format:
# (1) One sentence per line. These should ideally be actual sentences, not
# entire paragraphs or arbitrary spans of text. (Because we use the
# sentence boundaries for the "next sentence prediction" task).
# (2) Blank lines between documents. Document boundaries are needed so
# that the "next sentence prediction" task doesn't span between documents

In [ ]:
#!/usr/bin/env bash

# Read data_text_dir path from a config file.
CURDIR=$(cd $(dirname $0); pwd)
source <(sed -n '/^\[DATA\]/,/^\[/p' ${CURDIR}/../config.ini | grep TEXTDIR | sed 's/ *= */=/g')

#Text preprocessing

for FILE in $( find ${TEXTDIR} -name "wiki_*" ); do
    echo "Processing ${FILE}"
    sed -i -e '/^$/d; /<doc id/,+1d; s/<\/doc>//g' ${FILE}
    sed -i -e 's/ *$//g; s/。\([^」|)|）|"]\)/。\n\1/g; s/^[ ]*//g' ${FILE}
    sed -i -e '/^。/d' ${FILE}
    sed -i -e 's/\(.*\)/\L\1/' ${FILE}
done

# Concat all text files in each text directory.
for DIR in $( find ${TEXTDIR} -mindepth 1 -type d ); do
    echo "Processing ${DIR}"
    for f in $( find ${DIR} -name "wiki_*" ); do cat $f >> ${DIR}/all.txt; done 
done